<a href="https://colab.research.google.com/github/VickkiMars/NLP_Mastery/blob/main/Dense_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cohere tqdm Annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.4/388.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━

In [2]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from annoy import AnnoyIndex
from sklearn.metrics.pairwise import cosine_similarity

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
api_key = "..."
co = cohere.Client(api_key)

In [12]:
text = """
Back to the stature of David, as the armor-bearer and protector of Saul,
David was likely not below average in height.
Besides, Saul himself was head and shoulders above the average Israelite (1 Samuel 9:2).
From an archeological perspective, the remains of men in that area and at that time average 5’5”, thus Saul would have been around 6’.
If Samuel was impressed with the stature of David’s brother, then the genes in the sons of Jesse were certainly taller than the average citizen.
It is logical that Saul, a tall man of his day, would have selected a warrior of similar height to bear his armor.
Likewise, when David was to fight Goliath, Saul placed his own armor upon David.
This is not to be a comical situation: Saul, who was much bigger than the average male, would not have dreamt to try placing his oversized armor upon someone who some believe to be significantly shorter than average.
No, this fact asserts that Saul and David were indeed similar in stature.
David’s refusal of the armor was not because it was too big for him, but because he had “not proved them” (1 Samuel 17:39), that is, he had not trained with the king’s more protective type of armor, and it would have only hindered his action in fighting Goliath, for he was used to the common uniform.

Considering briefly the facts that David had wielded the sword of Goliath (which may have been bigger and heavier than most swords, though we can’t know that for sure) to cut off Goliath’s head, and David would later use Goliath’s sword for his own defense (1 Samuel 21:9), and considering that David struck dead both lions and bears (1 Samuel 17:34-36), the evidence surmounts that he was tremendously mighty, and most likely similar in stature to Saul, about 6’.

However, as to the stature of Goliath, I must go in the opposite direction.
I am for a shorter Goliath, and these are my reason:. Now, we all know that most of the English translations say that Goliath was “six cubits and a span” (1 Samuel 17:4), which is an astounding 9’9”.
However, the Septuagint, a translation of the old testament widely used in the first century by the church and quoted often in the new testament letters, says that Goliath was “four cubits and a span” (6’9”).
Now, Goliath is never called a giant in the scriptures, only in tales of David and Goliath.
The term giant (nephil) in the scriptures refers not to stature, but by definition, it refers to a bully, a tyrant, but this word is not used of Goliath or his family.
Later, Goliath’s family are called “giants” (rapha, 2 Samuel 21:22), which is a bit of a stretch, for the word “rapha” is defined as “tall”, or more literally “greater than average”.
Nonetheless, the Goliath that the first-century church read about was very tall, a giant of his day, standing nearly a foot and a half taller than the Israelites.
But where did the much taller Goliath (9’9”) originate from? For much of the history of the English translations, translators have relied predominately on the Masoretic text as the Hebrew authority for translating the old testament.
The Masoretic text is very accurate, but is at a disadvantage, being written far later than the old testament (around 1000 A.D).
In more recent years, the dead sea scrolls have been unearthed, which are 1,000 years older than the Masoretic text, and the scrolls reveal Goliath to be 6’9”.
And that’s not all! Josephus, the Jewish historian of the first century, recorded that Goliath was “a man of vast bulk, for he was of four cubits and a span in tallness: and had about him weapons suitable to the largeness of his body”.
Josephus has him at 6’9” too, but also addresses his “vast bulk”, so it wasn’t just that he was significantly taller than the men of his day, but he was very large and muscular all the way around; making him a terrifying opponent to Israel.
And, while the bible doesn’t really label him as a giant, the text does label him “champion”.
So, between his vast bulk, enormous weaponry, proven skill as a champion, and a height nearly a foot and a half above Israel, it is no wonder that Israel fears Goliath! So again, with the witness of Josephus, the dead sea scrolls, paired with the even earlier Septuagint version (200 B.C.), all of which record Goliath at 6’9”, it becomes much more likely that Goliath’s actual height was 6’9”, a deflating three feet shorter than most English accounts, but a great beast of a man in his day, even still in our day.
"""
texts = text.split('.')
texts = [t.strip(' \n') for t in texts]

In [13]:
response = co.embed(texts=texts,).embeddings
embeds = np.array(response)
print(embeds.shape)

(29, 4096)


In [14]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
for i, embed in enumerate(embeds):
    search_index.add_item(i, embed)
search_index.build(10)
search_index.save('search_index.ann')

True

In [22]:
def search(query:str) -> str:
  query_embed = co.embed(texts=[query]).embeddings
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0], n=3, include_distances=True)
  #print(similar_item_ids)
  res = pd.DataFrame(
      data={'texts':[texts[i] for i in similar_item_ids[0]],
            'distance':similar_item_ids[1]
            }
  )
  print(f"Query: '{query}' \nNearest neighbors:")
  return res, texts[similar_item_ids[0][0]]


In [24]:
q = "How tall was Goliath?"
search(q)

Query: 'How tall was Goliath?' 
Nearest neighbors:


(                                               texts  distance
 0  ), all of which record Goliath at 6’9”, it bec...  0.564970
 1  And that’s not all! Josephus, the Jewish histo...  0.581661
 2  Nonetheless, the Goliath that the first-centur...  0.626299,
 '), all of which record Goliath at 6’9”, it becomes much more likely that Goliath’s actual height was 6’9”, a deflating three feet shorter than most English accounts, but a great beast of a man in his day, even still in our day')